In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy

<IPython.core.display.Javascript object>

In [3]:
# Path to dump file and name
dumpfile = os.path.join('../data/dump/dump_knn_dump_file1')
breweries_path = os.path.join('../data/csv/breweries.csv')

beer_pickel_path = os.path.join('../data/dump/beer.pkl')

<IPython.core.display.Javascript object>

In [4]:
# Lets load our dump files
predictions,algo = dump.load(dumpfile)
breweries_df = pd.read_csv(breweries_path)
breweries_df=breweries_df.rename(columns = {'id':'brewery_id', 'name':'brewery_name'})
beers_df = pd.read_pickle(beer_pickel_path)

<IPython.core.display.Javascript object>

In [5]:
beers_df = pd.merge(beers_df, breweries_df, on='brewery_id')

<IPython.core.display.Javascript object>

In [6]:
beers_df = beers_df.drop(['notes', 'types', 'inner_ids'], axis=1)

<IPython.core.display.Javascript object>

In [7]:
beers_df.head()

,beer_id,score,name,style,brewery_id,brewery_name,city,state,country
0,5,3.387036,Amber,Vienna Lager,3,Abita Brewing Co.,Abita Springs,LA,US
1,6,3.627604,Turbodog,English Brown Ale,3,Abita Brewing Co.,Abita Springs,LA,US
2,7,3.187646,Purple Haze,Fruit and Field Beer,3,Abita Brewing Co.,Abita Springs,LA,US
3,1565,3.516667,Andygator,German Maibock,3,Abita Brewing Co.,Abita Springs,LA,US
4,39390,3.322879,Jockamo IPA,American IPA,3,Abita Brewing Co.,Abita Springs,LA,US


<IPython.core.display.Javascript object>

In [8]:
beers_df["beer_brewery"] = beers_df["name"] +"; "+ beers_df["brewery_name"]
beers_df.head()

,beer_id,score,name,style,brewery_id,brewery_name,city,state,country,beer_brewery
0,5,3.387036,Amber,Vienna Lager,3,Abita Brewing Co.,Abita Springs,LA,US,Amber; Abita Brewing Co.
1,6,3.627604,Turbodog,English Brown Ale,3,Abita Brewing Co.,Abita Springs,LA,US,Turbodog; Abita Brewing Co.
2,7,3.187646,Purple Haze,Fruit and Field Beer,3,Abita Brewing Co.,Abita Springs,LA,US,Purple Haze; Abita Brewing Co.
3,1565,3.516667,Andygator,German Maibock,3,Abita Brewing Co.,Abita Springs,LA,US,Andygator; Abita Brewing Co.
4,39390,3.322879,Jockamo IPA,American IPA,3,Abita Brewing Co.,Abita Springs,LA,US,Jockamo IPA; Abita Brewing Co.


<IPython.core.display.Javascript object>

In [9]:
def get_beer_brewery (beer_raw_id):
    beer_brewery = beers_df.loc[beers_df.beer_id==beer_raw_id,'beer_brewery'].values[0]
    return beer_brewery

def get_beer_raw_id (beer_name):
    beer_raw_id = beers_df.loc[beers_df.name==beer_name,'beer_id'].values[0]
    return beer_raw_id

def get_beer_style (beer_raw_id):
    beer_style = beers_df.loc[beers_df.beer_id==beer_raw_id,'style'].values[0]
    return beer_style

def get_beer_score_mean (beer_raw_id):
    score_mean = beers_df.loc[beers_df.beer_id==beer_raw_id,'score'].values[0]
    return score_mean

def get_beer_neighbors (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                  for inner_id in beer_neighbors)
    return(beer_neighbors)

def get_beer_recc_df (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                      for inner_id in beer_neighbors)
    beers_id_recc = []
    beer_brewery_recc =[]
    beer_style_recc = []
    beer_score_mean = []
    for beer in beer_neighbors:
        beers_id_recc.append(beer)
        beer_brewery_recc.append(get_beer_brewery(beer))
        beer_style_recc.append(get_beer_style(beer))
        beer_score_mean.append(get_beer_score_mean(beer))
    beer_reccomendations_df = pd.DataFrame(list(zip(beers_id_recc,
                                                    beer_brewery_recc,
                                                    beer_style_recc,
                                                    beer_score_mean)),
                                       columns=['beer_id', 'beer_brewery','style', 'score_mean'])
    return beer_reccomendations_df

<IPython.core.display.Javascript object>

In [10]:
trainset = algo.trainset

<IPython.core.display.Javascript object>

In [11]:
# Lets test some beers.  Enter a beer and use the predictions model to return 5 nearest neighbors
# K=10
beer_name = "Corona Extra"
beer_raw_id = get_beer_raw_id(beer_name)
print(f'The 10 nearest neighbors of {beer_name}, {get_beer_style(beer_raw_id)},\
 score = {get_beer_score_mean (beer_raw_id)} are:')
df = get_beer_recc_df (beer_raw_id)
df.head(10)

The 10 nearest neighbors of Corona Extra, American Adjunct Lager, score = 2.2439050847457627 are:


,beer_id,beer_brewery,style,score_mean
0,233,Corona Light; Grupo Modelo S.A. de C.V.,American Light Lager,1.870044
1,1320,Bud Light; Anheuser-Busch,American Light Lager,1.788794
2,41821,Bud Light Lime; Anheuser-Busch,American Light Lager,2.062840
3,2755,Cerveza Pacifico Clara; Grupo Modelo S.A. de C.V.,American Adjunct Lager,2.628512
4,332,Miller Lite; Miller Brewing Co.,American Light Lager,2.089922
5,2435,Beck's; Brauerei Beck & Co.,German Pilsner,2.711412
6,10071,Sapporo Premium Beer; Sapporo Breweries Ltd.,American Lager,2.800680
7,65,Budweiser; Anheuser-Busch,American Adjunct Lager,2.272213
8,1790,Peroni Nastro Azzurro; Birra Peroni Industrial...,European Pale Lager,2.736756
9,1351,Yuengling Traditional Lager; Yuengling Brewery,American Amber / Red Lager,3.312699


<IPython.core.display.Javascript object>

In [12]:
beers_df.to_pickle("../data/dump/beer_final1.pkl")

<IPython.core.display.Javascript object>